In [1]:
import csv, nltk, pickle, re, time
import pandas as pd
import numpy as np
from io import StringIO
from collections import Counter
from scipy import sparse
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

# need to use once to download nltk (natural language processing library) on your computer.
# nltk.download()

In [2]:
with open('outputs/train_neg_proc.pkl', 'rb') as f:
    neg_DF = pickle.load(f)
    
with open('outputs/train_pos_proc.pkl', 'rb') as f:
    pos_DF = pickle.load(f)
    
with open('outputs/test_data_proc.pkl', 'rb') as f:
    test_DF = pickle.load(f)

In [3]:
neg_DF.head()

,0,stemmed,lemmed,both
0,"[vinco, tresorpack, difficulty, object, disass...","[vinco, tresorpack, difficulti, object, disass...","[vinco, tresorpack, difficulty, object, disass...","[vinco, tresorpack, difficulti, object, disass..."
1,"[glad, dot, taks, tomorrow, thankful, startho]","[glad, dot, tak, tomorrow, thank, startho]","[glad, dot, taks, tomorrow, thankful, startho]","[glad, dot, tak, tomorrow, thank, startho]"
2,"[vs, celtics, regular, season, fucked, play, p...","[vs, celtic, regular, season, fuck, play, play...","[v, celtic, regular, season, fucked, play, pla...","[v, celtic, regular, season, fuck, play, playoff]"
3,"[could, actually, kill, girl, sorry]","[could, actual, kill, girl, sorri]","[could, actually, kill, girl, sorry]","[could, actual, kill, girl, sorri]"
4,"[find, hard, believe, im, afraid]","[find, hard, believ, im, afraid]","[find, hard, believe, im, afraid]","[find, hard, believ, im, afraid]"


<h2>TF-IDF</h2>

Create the vectorizer. We go with the idea that we do not want the words that appear in less than 5 tweets and in more than 80% of the tweets.

In [4]:
# create the vectoriser
vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf =True)

We now need to create a corpus. Our train set would both positive and negative, and our test set is, obviously, the unlabeled part.

To do this, we will append both negative and positive DF, then create a matrix of labels for them.

In [11]:
# put the list of words into a usable format
neg_DF = neg_DF["lemmed"]
pos_DF = pos_DF["lemmed"]
test_DF = test_DF["lemmed"]
neg_DF = pd.DataFrame(neg_DF)
pos_DF = pd.DataFrame(pos_DF)
test_DF = pd.DataFrame(test_DF)
neg_DF["lemmed"] = neg_DF.lemmed.apply(' '.join)
pos_DF["lemmed"] = pos_DF.lemmed.apply(' '.join)
test_DF["lemmed"] = test_DF.lemmed.apply(' '.join)

In [12]:
# we thus know that all the first ones are labeled as -1 and all the others as 1
all_labeled_DF = pd.concat([neg_DF, pos_DF])

In [13]:
# we create the labels
negs = len(neg_DF.index)
poss = len(pos_DF.index)
labels = np.zeros(negs+poss)
labels[0:negs]=-1
labels[negs:negs+poss]=1 

In [14]:
train_corpus_tf_idf = vectorizer.fit_transform(all_labeled_DF) 
test_corpus_tf_idf = vectorizer.transform(test_DF)

ValueError: max_df corresponds to < documents than min_df

In [ ]:
# create both models
model1 = LinearSVC() # SVM
model2 = MultinomialNB()

In [ ]:
# train on the given models
model1.fit(train_corpus_tf_idf,labels)
model2.fit(train_corpus_tf_idf,labels)

In [ ]:
# predictions
result1 = model1.predict(test_corpus_tf_idf)
result2 = model2.predict(test_corpus_tf_idf)

Result1 and result2 are the labels predicted for the tweets we got in the test corpus. This means we probably jsute have to transforme this into a csv as it is shown in the sample submission.

In [ ]:
# Converting it to integer for prediction csv
result1 = [int(x) for x in result1]
result2 = [int(x) for x in result2]

In [ ]:
svm_df = pd.DataFrame(result1)
svm_df['Id'] = svm_df.index + 1
svm_df['Prediction'] = svm_df[0]
svm_df = svm_df[['Id', 'Prediction']]
svm_df.to_csv('svm.csv', index=False)

In [ ]:
bayes_df = pd.DataFrame(result2)
bayes_df['Id'] = bayes_df.index + 1
bayes_df['Prediction'] = bayes_df[0]
bayes_df = bayes_df[['Id', 'Prediction']]
bayes_df.to_csv('bayes.csv', index=False)